<a href="https://colab.research.google.com/github/katterine2558/mri-tumor-detection-cnn/blob/main/mri_tumor_detection_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# 🔧 CONFIGURACIÓN DE ENTORNO COLAB PRO (GPU + OPTIMIZACIONES)
# ==========================================================
import os, gc, psutil, tensorflow as tf
from tensorflow.keras import mixed_precision

# ----------------------------------------------------------
# LIMPIEZA DE SESIÓN ANTERIOR
# ----------------------------------------------------------
gc.collect()
tf.keras.backend.clear_session()

# ----------------------------------------------------------
# INFORMACIÓN DEL HARDWARE
# ----------------------------------------------------------
try:
    gpu_info = !nvidia-smi --query-gpu=name,memory.total --format=csv,noheader
    print("GPU detectada:", gpu_info[0])
except:
    print("No se detectó GPU, verifica que el acelerador esté activado en Entorno > Configuración de ejecución.")

# ----------------------------------------------------------
# FORZAR USO DE GPU
# ----------------------------------------------------------
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    print(f"TensorFlow detecta {len(physical_gpus)} GPU(s): {physical_gpus}")
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        pass
else:
    print("No hay GPU activa, se usará CPU (entrenamiento más lento).")

# ----------------------------------------------------------
# ACTIVAR ENTRENAMIENTO DE PRECISIÓN MIXTA
# ----------------------------------------------------------
mixed_precision.set_global_policy('mixed_float16')
print("Política de precisión:", mixed_precision.global_policy())

# ----------------------------------------------------------
# MOSTRAR INFORMACIÓN DE RECURSOS
# ----------------------------------------------------------
print(f"Memoria RAM disponible: {psutil.virtual_memory().total / 1e9:.2f} GB")
print("Entorno configurado correctamente para máximo rendimiento.\n")

In [ ]:
#==========================================================
# LIBRARIES
#==========================================================
from google.colab import files
#!pip install -q kaggle #Descomentar si no está instalado Kaggle en el entorno
import os
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np
from collections import Counter
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Add, Activation,GlobalAveragePooling2D , Multiply, Reshape, DepthwiseConv2D, BatchNormalization ,GlobalAveragePooling2D, Resizing, Rescaling, RandomBrightness, RandomContrast, RandomRotation, RandomZoom, BatchNormalization, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import AUC
!pip install keras-tuner --upgrade
from keras_tuner.tuners import RandomSearch
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns
!nvidia-smi

#Ruta base
base_dir = '/content/mri_data'
#Ruta para almacenar split
split_dir = "/content/mri_data_split"
#Semilla
seed = 0;
#Tamaño del batch
batch_size = 16
#Image size
image_size = (260, 260)

print("Dispositivo:", tf.config.list_physical_devices('GPU'))
mixed_precision.set_global_policy('mixed_float16')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.debugging.set_log_device_placement(False)

# Carpeta donde guardas los resultados
tuner_dir = "tuning_results"

# Elimina la carpeta completa si existe
if os.path.exists(tuner_dir):
    shutil.rmtree(tuner_dir)

In [ ]:
#==========================================================
# DOWNLOAD DATASET
#==========================================================
def download_mri_data(base_dir:str):

  files.upload() #APIKey de Kaggle

  !mkdir -p ~/.kaggle
  !mv kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  # Descargar el dataset
  !mkdir -p /content/mri_data
  !kaggle datasets download -d rm1000/brain-tumor-mri-scans -p /content/mri_data --unzip

  # Listar categorías
  categories = os.listdir(base_dir)
  print("Categorías encontradas:", categories)

  # Contar imágenes por clase
  for c in categories:
      path = os.path.join(base_dir, c)
      print(f"{c}: {len(os.listdir(path))} imágenes")

  fig, axes = plt.subplots(1, 4, figsize=(16, 4))
  for i, c in enumerate(categories):
      folder = os.path.join(base_dir, c)
      img_name = random.choice(os.listdir(folder))
      img_path = os.path.join(folder, img_name)
      img = cv2.imread(img_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      axes[i].imshow(img)
      axes[i].set_title(c.capitalize())
      axes[i].axis('off')

  plt.tight_layout()
  plt.show()

#Descarga el conjunto de datos
download_mri_data(base_dir)

In [ ]:
#==========================================================
# IMAGE SIZE SCANNER
#==========================================================

def get_images_size(base_dir:str):

  all_sizes = []  # guardamos (ancho, alto)
  per_class_count = {}

  for cls in os.listdir(base_dir):
      cls_path = os.path.join(base_dir, cls)
      if not os.path.isdir(cls_path):
          continue

      imgs = os.listdir(cls_path)
      per_class_count[cls] = len(imgs)

      # vamos a mirar hasta 20 imágenes por clase (para no leer miles)
      for img_name in imgs[:20]:
          img_path = os.path.join(cls_path, img_name)
          img = cv2.imread(img_path)  # esto lee en BGR
          if img is None:
              # a veces hay archivos raros tipo .txt o corruptos
              continue
          h, w, ch = img.shape  # alto, ancho, canales
          all_sizes.append((w, h, ch))


  print("\nEjemplos de tamaños (ancho x alto x canales) en las primeras imágenes revisadas:")
  for s in all_sizes[:10]:
      print(" ", s)

  # estadísticas más globales
  widths = [s[0] for s in all_sizes]
  heights = [s[1] for s in all_sizes]
  channels = [s[2] for s in all_sizes]

  print("\nEstadísticas:")
  print(f"  Ancho min / max / promedio: {min(widths)} / {max(widths)} / {np.mean(widths):.1f}")
  print(f"  Alto  min / max / promedio: {min(heights)} / {max(heights)} / {np.mean(heights):.1f}")

  print("\nCanales más comunes (1=gris, 3=RGB):")
  print(Counter(channels))

#Obtiene el tamaño de las imágenes
get_images_size(base_dir)

In [ ]:
# ==========================================================
# TRAIN-VAL-TEST DIVISION (70-20-10)
# ==========================================================

def traint_val_test_split(base_dir:str, split_dir:str):
    # Crear carpetas destino
    for split in ['train', 'val', 'test']:
        for cls in os.listdir(base_dir):
            os.makedirs(os.path.join(split_dir, split, cls), exist_ok=True)

    # Ratios
    train_ratio = 0.7
    val_ratio = 0.2
    test_ratio = 0.1

    # Procesar cada clase
    for cls in os.listdir(base_dir):
        cls_path = os.path.join(base_dir, cls)
        images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.png','.jpg','.jpeg'))]

        # Dividir train / val / test
        train_files, temp_files = train_test_split(images, test_size=(1-train_ratio), random_state=seed)
        val_size = val_ratio / (val_ratio + test_ratio)
        val_files, test_files = train_test_split(temp_files, test_size=(1-val_size), random_state=seed)

        # Copiar archivos
        for fname in train_files:
            shutil.copy(os.path.join(cls_path, fname), os.path.join(split_dir, 'train', cls, fname))
        for fname in val_files:
            shutil.copy(os.path.join(cls_path, fname), os.path.join(split_dir, 'val', cls, fname))
        for fname in test_files:
            shutil.copy(os.path.join(cls_path, fname), os.path.join(split_dir, 'test', cls, fname))

    print("División completada con semilla =", seed)
    print("Estructura creada en:", split_dir)

    for split in ['train', 'val', 'test']:
        print(f"\n * {split.upper()}")
        for cls in os.listdir(os.path.join(split_dir, split)):
            count = len(os.listdir(os.path.join(split_dir, split, cls)))
            print(f"  {cls}: {count} imágenes")


# Ejecutar división solo una vez
traint_val_test_split(base_dir, split_dir)

# ==========================================================
# CARGA DE LOS DATASETS
# ==========================================================
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    f"{split_dir}/train",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    f"{split_dir}/val",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

test_ds_raw = tf.keras.utils.image_dataset_from_directory(
    f"{split_dir}/test",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False,
    seed=seed
)

AUTOTUNE = tf.data.AUTOTUNE

def prepare_dataset(ds, shuffle=False):
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = prepare_dataset(train_ds_raw, shuffle=True)
val_ds   = prepare_dataset(val_ds_raw)
test_ds  = prepare_dataset(test_ds_raw)

print("Datasets listos y optimizados para entrenamiento.")

In [ ]:
# ==========================================================
# TRAIN DATA AUGMENTATION
# ==========================================================

def data_augmentation(split_dir:str, seed:int, batch_size:int, image_size:tuple):
    # Cargar imágenes (se necesita tamaño fijo)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        f'{split_dir}/train',
        batch_size=batch_size,
        shuffle=True,
        seed=seed,
        image_size=image_size
    )

    class_names = train_ds.class_names
    print("Clases detectadas:", class_names)

    # Crear mapeo explícito de índice → nombre
    label_map = {i: name for i, name in enumerate(class_names)}
    print("\nMapeo de etiquetas:")
    for key, value in label_map.items():
        print(f"{key} → {value}")

    # Definir Data Augmentation
    data_augmentation = tf.keras.Sequential([
        RandomRotation(0.05),
        RandomZoom(0.1),
        RandomContrast(0.1),
        RandomBrightness(0.1),
    ])

    # Aplicar aumentación
    train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

    print("\n Data augmentation aplicada al conjunto de entrenamiento.")
    return train_ds, data_augmentation, class_names, label_map


def plot_data_augmentation(train_ds, data_augmentation):
    for images, labels in train_ds.take(1):
        plt.figure(figsize=(10, 10))
        for i in range(9):
            augmented = data_augmentation(images, training=True)
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(augmented[i].numpy().astype("uint8"))
            plt.axis("off")
        plt.suptitle("Ejemplos de Data Augmentation", fontsize=14)
        plt.show()
        break


# Ejecutar data augmentation
train_ds, data_aug, class_names, label_map = data_augmentation(split_dir, seed, batch_size, image_size)
plot_data_augmentation(train_ds, data_aug)

In [ ]:
# ==========================================================
# TRAIN LABEL VERIFICATION
# ==========================================================

def get_label(train_ds, class_names, label_map):

    # Mostrar forma y etiquetas del primer batch
    for images, labels in train_ds.take(1):
        print("\nShape de las imágenes:", images.shape)

        # Mostrar 9 imágenes con sus nombres de clase
        plt.figure(figsize=(10, 10))
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            label_idx = labels[i].numpy()
            plt.title(label_map[label_idx])
            plt.axis("off")
        plt.suptitle("Etiquetas visualizadas con nombres", fontsize=14)
        plt.show()
        break


# Ejecutar
get_label(train_ds, class_names, label_map)

for images, labels in val_ds.take(1):
    print("Batch shape:", images.shape)
    print("Labels shape:", labels.shape)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ==========================================================
# ACTIVACIÓN MISH
# ==========================================================
@tf.keras.utils.register_keras_serializable()
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))


# ==========================================================
# BLOQUE MBConv MEJORADO (con DropConnect)
# ==========================================================
def MBConvMRI(inputs, out_channels, expand_ratio, stride, se_ratio=0.25, drop_rate=0.2):
    in_channels = inputs.shape[-1]
    shortcut = inputs
    x = inputs

    expanded = int(in_channels * expand_ratio)
    if expand_ratio != 1:
        x = layers.Conv2D(expanded, 1, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(mish)(x)

    # Depthwise separable
    x = layers.DepthwiseConv2D(3, strides=stride, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    # Squeeze and Excitation
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Reshape((1,1,expanded))(se)
    se = layers.Conv2D(int(expanded * se_ratio), 1, activation=mish, padding='same')(se)
    se = layers.Conv2D(expanded, 1, activation='sigmoid', padding='same')(se)
    x = layers.Multiply()([x, se])

    # Proyección
    x = layers.Conv2D(out_channels, 1, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
    x = layers.BatchNormalization()(x)

    # Residual con DropConnect
    if stride == 1 and in_channels == out_channels:
        if drop_rate > 0:
            x = layers.Dropout(drop_rate)(x)
        x = layers.Add()([x, shortcut])
    return x


# ==========================================================
# MODELO EfficientMRI-Net v2
# ==========================================================
def build_efficient_mri_net_v2(num_classes=4, input_shape=(image_size[0], image_size[1], 3)):
    inputs = layers.Input(shape=input_shape)

    # -----------------------------
    # Data augmentation integrado
    # -----------------------------
    x = layers.RandomFlip("horizontal")(inputs)
    x = layers.RandomRotation(0.08)(x)
    x = layers.RandomZoom(0.1)(x)
    x = layers.RandomContrast(0.2)(x)
    x = layers.Rescaling(1./255)(x)

    # -----------------------------
    # Bloque inicial
    # -----------------------------
    x = layers.Conv2D(48, 3, strides=2, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    # -----------------------------
    # Bloques MBConv
    # -----------------------------
    x = MBConvMRI(x, 16, 1, 1, drop_rate=0.1)
    x = MBConvMRI(x, 24, 3, 2, drop_rate=0.1)
    x = MBConvMRI(x, 24, 3, 1, drop_rate=0.1)
    x = MBConvMRI(x, 40, 4, 2, drop_rate=0.15)
    x = MBConvMRI(x, 40, 4, 1, drop_rate=0.15)
    x = MBConvMRI(x, 80, 4, 2, drop_rate=0.2)
    x = MBConvMRI(x, 80, 4, 1, drop_rate=0.2)
    x = MBConvMRI(x, 112, 4, 1, drop_rate=0.25)
    x = MBConvMRI(x, 160, 4, 2, drop_rate=0.25)
    x = MBConvMRI(x, 320, 4, 1, drop_rate=0.3)

    # -----------------------------
    # Head final
    # -----------------------------
    x = layers.Conv2D(1280, 1, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = Model(inputs, outputs, name="EfficientMRI_Net_v2")
    return model


# ==========================================================
# COMPILACIÓN
# ==========================================================
model = build_efficient_mri_net_v2(num_classes=4, input_shape=(image_size[0], image_size[1],3))
optimizer = AdamW(learning_rate=8e-4, weight_decay=1e-5)
loss_fn = SparseCategoricalCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)

# ==========================================================
# CALLBACKS
# ==========================================================
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('EfficientMRI_Net_v2.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

model.summary()


# ==========================================================
# ENTRNEAMIENTO
# ==========================================================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=80,
    batch_size=batch_size,
    callbacks=[reduce_lr, early_stop, checkpoint],
    verbose=1
)


In [ ]:
# ==========================================================
# GRÁFICAS DE ENTRENAMIENTO Y VALIDACIÓN
# ==========================================================
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(acc) + 1)

plt.figure(figsize=(14,5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, 'b-', label='Entrenamiento')
plt.plot(epochs_range, val_acc, 'r--', label='Validación')
plt.title('Evolución de la Accuracy')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, 'b-', label='Entrenamiento')
plt.plot(epochs_range, val_loss, 'r--', label='Validación')
plt.title('Evolución de la Pérdida (Loss)')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

plt.show()

In [ ]:
# ==========================================================
# EVALUACIÓN FINAL EN TEST
# ==========================================================
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")
print(f"📉 Test Loss: {test_loss:.4f}")


In [ ]:
# ==========================================================
# PREDICCIONES Y REPORTES
# ==========================================================
# Obtener etiquetas verdaderas y predichas
y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Etiquetas de las clases
class_names = test_ds.class_names
print("\n🧾 Reporte de Clasificación:\n")
print(classification_report(y_true, y_pred, target_names=class_names, digits=2))

# ==========================================================
# MATRIZ DE CONFUSIÓN
# ==========================================================
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Matriz de Confusión - EfficientMRI-Net")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()
